In [ ]:
import glob
import numpy as np
import cv2
import h5py
import os
import tables

In [ ]:
hdf5_save_path = "../data/pose_raw.hdf5"
content_path = "../../../precompiled_data/mpii_pose/images/"
data_path = "../data/dataset_updated.txt"

In [ ]:
lines = [line.rstrip('\n') for line in open(data_path)]

In [ ]:
print(len(lines))

In [ ]:
content_num = len(lines)
HEIGHT = 128
WIDTH = 128
CHANNELS = 3
SHAPE = (0,HEIGHT, WIDTH, CHANNELS)
img_dtype = tables.UInt8Atom()

In [ ]:
hdf5_file = tables.open_file(hdf5_save_path, mode='w')
pose_stick = hdf5_file.create_earray(hdf5_file.root, 'pose_stick', img_dtype, shape=SHAPE)

In [ ]:
#init_array = [1.0,grid_coord_x,grid_coord_y,0.0625,0.0625]
#grid_data[0,1,1,:] = init_array.extend(np.identity(16)[3,:])
#print(init_array)
#grid_data = np.zeros((content_num, 16, 16, 21),dtype=np.float32)

In [ ]:
#(0 - r ankle, 1 - r knee, 2 - r hip, 3 - l hip, 4 - l knee, 5 - l ankle, 6 - pelvis, 7 - thorax, 8 - upper neck, 
#9 - head top, 10 - r wrist, 11 - r elbow, 12 - r shoulder, 13 - l shoulder, 14 - l elbow, 15 - l wrist)
#0 - red, 1 - green, 2 - blue, 3 - white, 4 - light yellow, 5 - grey 6 - yellow 7 - orange 8 - violet
#9 - maroon 10 - indigo  11 - brown 12 - pink 13 - light blue 14 - dark green 15 - dark blue

color_val_array = [[255,0,0],[0,255,0],[0,0,255],[255,255,255],[127,127,0],[127,127,127],[255,255,0],
                  [255,165,0],[238,130,238],[128,0,0],[75,0,130],[165,42,42],[255,192,203],[78,152,160],
                  [0,128,0],[0,0,128]]

print(color_val_array[3])


In [ ]:
i = 0
for line in lines:
    line = line.split(" ")
    image_data = line[0]
    image_data = image_data[0:13]
    address = content_path + image_data
    img = cv2.imread(address)
    img_black = np.zeros((128,128,3), np.uint8)
   
    #split drawing to legs torso and hands
    #legs
    for item in range(5, 17, 2):
        if (item + 3 < 17):
            x1, y1 = float(line[item]), float(line[item + 1])
            x2, y2 = float(line[item + 2]), float(line[item + 3])

            x1 = x1 * 128.0/img.shape[1]
            x2 = x2 * 128.0/img.shape[1]
            y1 = y1 * 128.0/img.shape[0]
            y2 = y2 * 128.0/img.shape[0]

            if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
                continue

            #print("x1 " + str(x1) + " y1 " + str(y1) + " x2 " + str(x2) + " y2 " + str(y2))
            cv2.line(img_black,(int(x1),int(y1)),(int(x2),int(y2)),color_val_array[0],2)
    
    #torso
    for item in range(17, 25, 2):
        if (item + 3 < 25):
            x1, y1 = float(line[item]), float(line[item + 1])
            x2, y2 = float(line[item + 2]), float(line[item + 3])

            x1 = x1 * 128.0/img.shape[1]
            x2 = x2 * 128.0/img.shape[1]
            y1 = y1 * 128.0/img.shape[0]
            y2 = y2 * 128.0/img.shape[0]

            if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
                continue

            cv2.line(img_black,(int(x1),int(y1)),(int(x2),int(y2)),color_val_array[1],2)

     #hands       
    for item in range(25, 37, 2):
        if (item + 3 < 37):
            x1, y1 = float(line[item]), float(line[item + 1])
            x2, y2 = float(line[item + 2]), float(line[item + 3])

            x1 = x1 * 128.0/img.shape[1]
            x2 = x2 * 128.0/img.shape[1]
            y1 = y1 * 128.0/img.shape[0]
            y2 = y2 * 128.0/img.shape[0]

            if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
                continue

            cv2.line(img_black,(int(x1),int(y1)),(int(x2),int(y2)),color_val_array[2],2) 
         
    
    i = i + 1
        
    pose_stick.append(img_black[None])

In [ ]:
hdf5_file.close()

In [ ]:
hdf5_path = "../data/pose_raw.hdf5"
hdf5_file = tables.open_file(hdf5_path, mode='r')
pose_num = hdf5_file.root.pose_stick.shape[0]
print(" content num is " + str(pose_num))

In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
imshow(hdf5_file.root.pose_stick[987,:,:,:])

In [ ]:
#(0 - r ankle, 1 - r knee, 2 - r hip, 3 - l hip, 4 - l knee, 5 - l ankle, 6 - pelvis, 7 - thorax, 8 - upper neck, 
#9 - head top, 10 - r wrist, 11 - r elbow, 12 - r shoulder, 13 - l shoulder, 14 - l elbow, 15 - l wrist)

index = 29
#img = img_array[index,:,:,:]
content_path = "../../../precompiled_data/mpii_pose/images/"
img = cv2.imread(content_path + "036636184.jpg")
img_black = np.zeros((128,128,3), np.uint8)
line = lines[index]
line = line.split(" ")

#split drawing to legs torso and hands
color_index = 0
#legs
for item in range(5, 17, 2):
    if (item + 3 < 17):
        x1, y1 = float(line[item]), float(line[item + 1])
        x2, y2 = float(line[item + 2]), float(line[item + 3])
        
        x1 = x1 * 128.0/img.shape[1]
        x2 = x2 * 128.0/img.shape[1]
        y1 = y1 * 128.0/img.shape[0]
        y2 = y2 * 128.0/img.shape[0]
        
        if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
            #color_index = color_index + 1
            continue
        
        print("x1 " + str(x1) + " y1 " + str(y1) + " x2 " + str(x2) + " y2 " + str(y2))
        cv2.line(img_black,(int(x1),int(y1)),(int(x2),int(y2)),color_val_array[0],2)
        #color_index = color_index + 1 

for item in range(17, 25, 2):
    if (item + 3 < 25):
        x1, y1 = float(line[item]), float(line[item + 1])
        x2, y2 = float(line[item + 2]), float(line[item + 3])
        
        x1 = x1 * 128.0/img.shape[1]
        x2 = x2 * 128.0/img.shape[1]
        y1 = y1 * 128.0/img.shape[0]
        y2 = y2 * 128.0/img.shape[0]
        
        if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
            #color_index = color_index + 1
            continue
        
        cv2.line(img_black,(int(x1),int(y1)),(int(x2),int(y2)),color_val_array[1],2)
        #color_index = color_index + 1 

for item in range(25, 37, 2):
    if (item + 3 < 37):
        x1, y1 = float(line[item]), float(line[item + 1])
        x2, y2 = float(line[item + 2]), float(line[item + 3])
        
        x1 = x1 * 128.0/img.shape[1]
        x2 = x2 * 128.0/img.shape[1]
        y1 = y1 * 128.0/img.shape[0]
        y2 = y2 * 128.0/img.shape[0]
        
        if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
            #color_index = color_index + 1
            continue
        
        cv2.line(img_black,(int(x1),int(y1)),(int(x2),int(y2)),color_val_array[2],2) 
        #color_index = color_index + 1

imshow(img_black)

In [ ]:
imshow(img)